In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Load dataset

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [2]:
# Convolutional Layer

class CustomConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1):
        super(CustomConv2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation

        
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.zeros(out_channels))

    def forward(self, x):
        unfolded = torch.nn.functional.unfold(x, self.kernel_size, padding=self.padding, stride=self.stride, dilation=self.dilation)
        unfolded = unfolded.transpose(1, 2).contiguous().view(-1, unfolded.size(1))
        weight = self.weight.view(self.weight.size(0), -1)

        output = unfolded.matmul(weight.t()) + self.bias
        output = output.view(x.size(0), self.weight.size(0), (x.size(2) - self.kernel_size + 2 * self.padding) // self.stride + 1,
                             (x.size(3) - self.kernel_size + 2 * self.padding) // self.stride + 1)
        return output

    
# Max Pooling Layer    
    
class CustomMaxPool2d(nn.Module):
    def __init__(self, kernel_size, stride=None, padding=0, dilation=1):
        super(CustomMaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride if stride is not None else kernel_size
        self.padding = padding
        self.dilation = dilation

    def forward(self, x):
        unfolded = torch.nn.functional.unfold(x, self.kernel_size, padding=self.padding, stride=self.stride, dilation=self.dilation)
        unfolded = unfolded.view(x.size(0), x.size(1), -1, x.size(2), x.size(3))

        output, _ = unfolded.max(dim=2)
        return output

    
    
# Neural Network
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.conv1 = CustomConv2d(1, 64, kernel_size=3, stride=1, padding=0, dilation=2)
        self.pool1 = CustomMaxPool2d(kernel_size=3, stride=2, dilation=2)
        self.conv2 = CustomConv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=2)
        self.pool2 = CustomMaxPool2d(kernel_size=3, stride=2, dilation=2)
        self.fc1 = nn.Linear(128 * 5 * 5, 256) 
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


custom_net = CustomNet()


input_data = torch.randn(64, 1, 28, 28)  

# Forward pass
output = custom_net(input_data)
print(output.shape) 

RuntimeError: shape '[64, 64, 26, 26]' is invalid for input of size 2359296

In [ ]:
custom_net = CustomNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_net.parameters(), lr=0.001)

# Training loop

epochs = 5

for epoch in range(epochs):
    custom_net.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = custom_net(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Epoch {} Batch {}/{} Loss: {:.6f}'.format(epoch, batch_idx, len(train_loader), loss.item()))

# Testing the trained model

custom_net.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        output = custom_net(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy on the test set: {:.2f}%'.format(100 * correct / tota